## Goal
The main purpose of this notebook is to develope the code to read in phenotypes in desired format.
At the end, I should wrap it up as the function.

In [2]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://10.151.24.211:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /Users/yanyul/Documents/repo/github/ptrs-ukb/notebook/hail-20191213-0830-0.2.28-61941242c15d.log


In [1]:
import pandas as pd
import numpy as np

In [7]:
covar_names = 'age_recruitment,sex,pc1,pc2'
pheno_names = 'ht,mcv,mch'
indiv_id = 'eid'
int_names = 'age_recruitment,sex'
str_names = 'eid'

In [60]:
def names_to_list(names):
    return names.split(',')
def get_dtype_dic(int_names, str_names, all_names):
    out_dic = { a : np.float for a in all_names }
    for i in int_names:
        out_dic[i] = np.int
    for i in str_names:
        out_dic[i] = str
    return out_dic
def read_and_split_phenotype_csv(csv_path, pheno_names, covar_names, indiv_id, int_names, str_names):
    # read in the phenotype table prepared from ukbREST output (in CSV format)
    # split and return two tables
    # 1. covariate table
    # 2. trait table
    covar_list = names_to_list(covar_names)
    pheno_list = names_to_list(pheno_names)
    all_colnames = covar_list + pheno_list + names_to_list(indiv_id)
    type_dic = get_dtype_dic(names_to_list(int_names), names_to_list(str_names), all_colnames)
    pheno = pd.read_csv(csv_path, dtype = type_dic, usecols = list(type_dic.keys())).rename(columns = {indiv_id : 's'})
    covar_table = pheno[covar_list + ['s']]
    trait_table = pheno[pheno_list + ['s']]
    return covar_table, trait_table
def read_indiv_list(file_path):
    # read in the file from file_path
    # and return the first columns as a list of string 
    indiv_list = pd.read_csv(file_path, sep = '\s+', dtype = str)
    return indiv_list.iloc[:,0].to_list()
def subset_by_col(df, colname, target_list):
    return df[df[colname].isin(target_list)]


In [31]:
covar, trait = read_and_split_phenotype_csv(
    '../output/query_phenotypes_cleaned_up.csv',
    pheno_names = pheno_names,
    covar_names = covar_names,
    indiv_id = indiv_id,
    int_names = int_names,
    str_names = str_names
)

#### Now that we've loaded in the full covariate and trait tables
Here we start to loop over all subsets and build the "list of lists" for traits.

In [74]:
subset_indiv_list = read_indiv_list('../output/data_split/British-training-1.txt')
sub_trait = subset_by_col(trait, 's', subset_indiv_list)
sub_trait = hl.Table.from_pandas(sub_trait, key = 's')
sub_trait = sub_trait.repartition(40)

2019-12-13 09:17:56 Hail: INFO: Ordering unsorted dataset with network shuffle


TypeError: repartition() missing 1 required positional argument: 'n'

In [4]:
pheno

,eid,age_recruitment,sex,pc1,pc2,pc3,pc4,pc5,pc6,pc7,...,ht,mcv,mch,mchc,platelet,lymphocyte,monocyte,neutrophil,eosinophil,basophil
0,1000051,61,0,-12.26880,3.80055,-4.335700,-1.224900,2.81216,-1.841370,2.540660,...,38.00,98.48,33.79,34.31,211.6,2.05,0.36,3.43,0.08,0.02
1,1000206,57,0,-10.71030,4.52891,-2.895460,1.268830,1.56004,1.973210,0.592195,...,37.50,87.10,30.50,35.00,405.0,3.80,1.10,5.60,0.20,0.10
2,1000417,46,0,-15.59890,2.70182,-4.182400,3.895720,-7.52412,1.653820,-0.641735,...,39.71,94.25,34.18,36.27,317.4,2.02,0.74,0.77,0.16,0.03
3,1000686,51,0,-9.87922,3.18668,-3.122890,0.831482,1.74031,1.453750,0.771818,...,39.62,93.71,32.33,34.50,253.5,1.98,0.46,4.60,0.29,0.04
4,1000704,54,0,61.49890,-99.07530,75.386300,8.585040,0.10777,6.460660,0.608611,...,35.48,88.67,30.01,33.85,362.6,2.76,0.60,4.34,0.75,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365421,6025471,65,1,-13.02600,7.03332,-4.015890,3.952580,7.02332,-1.749340,1.786230,...,42.90,93.10,32.40,34.80,218.0,1.80,0.40,3.00,0.20,0.10
365422,6025941,55,1,-13.04580,3.86199,-3.925010,-1.403990,-4.59881,0.398345,2.350770,...,42.20,89.16,29.51,33.10,261.4,2.34,0.53,3.04,0.08,0.04
365423,6026056,56,1,-13.37450,4.22199,-0.001266,2.544730,4.77374,1.063770,-0.929151,...,39.80,89.60,31.20,34.90,306.0,1.30,0.50,5.50,0.20,0.00
365424,6026321,60,0,-14.68390,5.09672,-0.795096,-0.266185,-6.53859,0.181480,-1.407390,...,38.42,91.43,31.91,34.90,204.9,2.40,0.60,6.83,0.33,0.04
